# Lab 5

You are tasked with evaluating card counting strategies for black jack. In order to do so, you will use object oriented programming to create a playable casino style black jack game where a computer dealer plays against $n$ computer players and possibily one human player. If you don't know the rules of blackjack or card counting, please google it. 

A few requirements:
* The game should utilize multiple 52-card decks. Typically the game is played with 6 decks.
* Players should have chips.
* Dealer's actions are predefined by rules of the game (typically hit on 16). 
* The players should be aware of all shown cards so that they can count cards.
* Each player could have a different strategy.
* The system should allow you to play large numbers of games, study the outcomes, and compare average winnings per hand rate for different strategies.

1. Begin by creating a classes to represent cards and decks. The deck should support more than one 52-card set. The deck should allow you to shuffle and draw cards. Include a "plastic" card, placed randomly in the deck. Later, when the plastic card is dealt, shuffle the cards before the next deal.

In [1]:
import random 

class Card:
    def __init__(self, rank):
        self.rank = rank

    def __str__(self):
        return f"{self.rank}"

class Deck:
    def __init__(self, num_decks=1):
        ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']

        self.cards = [Card(rank) for rank in ranks] * num_decks
        self.plastic_card_position = random.randint(20, 30)  # Place plastic card randomly between 20 and 30 cards remaining
        self.shuffle()

    def shuffle(self):
        random.shuffle(self.cards)

    def draw_card(self):
        return self.cards.pop(0)

    def insert_plastic_card(self):
        self.cards.insert(self.plastic_card_position, "Plastic Card")

class Hand:
    def __init__(self):
        self.cards = []

    def add_card(self, card):
        self.cards.append(card)

    def get_value(self):
        value = 0
        num_aces = 0

        for card in self.cards:
            if card.rank.isnumeric():
                value += int(card.rank)
            elif card.rank in ['J', 'Q', 'K']:
                value += 10
            elif card.rank == 'A':
                value += 11
                num_aces += 1

        # Adjust value for aces
        while value > 21 and num_aces:
            value -= 10
            num_aces -= 1

        return value
    

2. Now design your game on a UML diagram. You may want to create classes to represent, players, a hand, and/or the game. As you work through the lab, update your UML diagram. At the end of the lab, submit your diagram (as pdf file) along with your notebook. 

In [18]:
class Player:
    def __init__(self, name, initial_chips):
        self.name = name
        self.hand = Hand()
        self.chips = initial_chips
        
    def show_hand(self):
        print(f"{self.name}'s Hand: {' '.join(str(card) for card in self.hand.cards)}")

class ComputerPlayer(Player):
    def __init__(self, name, initial_chips):
        super().__init__(name, initial_chips)

    def show_hand(self):
        print(f"{self.name}'s Hand: {' '.join(str(card) for card in self.hand.cards)}") 

3. Begin with implementing the skeleton (ie define data members and methods/functions, but do not code the logic) of the classes in your UML diagram.

4. Complete the implementation by coding the logic of all functions. For now, just implement the dealer player and human player.

In [10]:
class BlackjackGame:
    def __init__(self, num_players=1):
        self.deck = Deck()
        self.players = [Player(f"Player {i+1}") for i in range(num_players)]
        self.computer_player = ComputerPlayer("Computer Dealer")

    def deal_initial_cards(self):
        for _ in range(2):
            for player in self.players + [self.computer_player]:
                player.hand.add_card(self.deck.draw_card())

    def play(self):
        self.deal_initial_cards()

        for player in self.players + [self.computer_player]:
            player.show_hand()
    
    def player_turn(self, player):
        while True:
            action = input(f"{player.name}, do you want to 'hit' or 'stand'? ").lower()
            if action == 'hit':
                player.hand.add_card(self.deck.draw_card())
                player.show_hand()
                
                if player.hand.get_value() > 21:
                    print(f"{player.name} busts! You have more than 21 points.")
                    break
            elif action == 'stand':
                player.show_hand()
                break
            else:
                print("Invalid input. Please enter 'hit' or 'stand'.")

    def computer_turn(self):
        while self.computer_player.hand.get_value() < 17:
            self.computer_player.hand.add_card(self.deck.draw_card())

    def play(self):
        self.deal_initial_cards()
        
        for player in self.players + [self.computer_player]:
            player.show_hand()

        for player in self.players:
            self.player_turn(player)

        # Computer's turn
        self.computer_turn()

        # Determine the winner
        for player in self.players:
            if player.hand.get_value() > 21:
                player.show_hand()
                print(f"{player.name} busted! Computer wins.")
            elif self.computer_player.hand.get_value() > 21:
                player.show_hand()
                print(f"Computer busted! {player.name} wins.")
            elif player.hand.get_value() > self.computer_player.hand.get_value():
                player.show_hand()
                print(f"{player.name} wins!")
            elif player.hand.get_value() < self.computer_player.hand.get_value():
                player.show_hand()
                print(f"Computer wins. {player.name}, better luck next time!")
            else:
                player.show_hand()
                print(f"{player.name} and Computer have a tie!")

# Let's test the game
game = BlackjackGame(num_players=1)
game.play()

Player 1's Hand: 7 4
Computer Dealer's Hand: 8 J
Player 1, do you want to 'hit' or 'stand'? hit
Player 1's Hand: 7 4 10
Player 1, do you want to 'hit' or 'stand'? stand
Player 1's Hand: 7 4 10
Player 1's Hand: 7 4 10
Player 1 wins!


5.  Test. Demonstrate game play. For example, create a game of several dealer players and show that the game is functional through several rounds.

In [11]:
game = BlackjackGame(num_players=2)
game.play()

Player 1's Hand: 3 5
Player 2's Hand: A J
Computer Dealer's Hand: 4 9
Player 1, do you want to 'hit' or 'stand'? hit
Player 1's Hand: 3 5 7
Player 1, do you want to 'hit' or 'stand'? stand
Player 1's Hand: 3 5 7
Player 2, do you want to 'hit' or 'stand'? stand
Player 2's Hand: A J
Player 1's Hand: 3 5 7
Computer busted! Player 1 wins.
Player 2's Hand: A J
Computer busted! Player 2 wins.


6. Implement a new player with the following strategy:

    * Assign each card a value: 
        * Cards 2 to 6 are +1 
        * Cards 7 to 9 are 0 
        * Cards 10 through Ace are -1
    * Compute the sum of the values for all cards seen so far.
    * Hit if sum is very negative, stay if sum is very positive. Select a threshold for hit/stay, e.g. 0 or -2.  

In [12]:
class CardCounterPlayer(Player):
    def __init__(self, name, threshold=-2):
        super().__init__(name)
        self.threshold = threshold
        self.card_count = 0

    def update_card_count(self, card):
        if card.rank in ['2', '3', '4', '5', '6']:
            self.card_count += 1
        elif card.rank in ['10', 'J', 'Q', 'K', 'A']:
            self.card_count -= 1

    def decide_action(self):
        if self.card_count <= self.threshold:
            return 'hit'
        else:
            return 'stand'

class BlackjackGame:
    def __init__(self, num_players=1):
        self.deck = Deck()
        self.players = [Player(f"Player {i+1}") for i in range(num_players)]
        self.computer_player = ComputerPlayer("Computer Dealer")

    def deal_initial_cards(self):
        for _ in range(2):
            for player in self.players + [self.computer_player]:
                player.hand.add_card(self.deck.draw_card())

    def play(self):
        self.deal_initial_cards()

        for player in self.players + [self.computer_player]:
            player.show_hand()
    
    def player_turn(self, player):
        while True:
            action = input(f"{player.name}, do you want to 'hit' or 'stand'? ").lower()
            if action == 'hit':
                player.hand.add_card(self.deck.draw_card())
                player.show_hand()
                
                if player.hand.get_value() > 21:
                    print(f"{player.name} busts! You have more than 21 points.")
                    break
            elif action == 'stand':
                player.show_hand()
                break
            else:
                print("Invalid input. Please enter 'hit' or 'stand'.")

    def computer_turn(self):
        while self.computer_player.hand.get_value() < 17:
            self.computer_player.hand.add_card(self.deck.draw_card())

    def card_counter_player_turn(self, player):
        while True:
            player.show_hand()
            action = player.decide_action()

            if action == 'hit':
                player.hand.add_card(self.deck.draw_card())
                self.computer_player.hand.add_card(self.deck.draw_card())  # Simulate the dealer's turn
                player.update_card_count(player.hand.cards[-1])
                if player.hand.get_value() > 21:
                    print(f"{player.name} busts! You have more than 21 points.")
                    break
            elif action == 'stand':
                break

    def play_with_card_counter(self):
        self.deal_initial_cards()

        for player in self.players:
            if isinstance(player, CardCounterPlayer):
                self.card_counter_player_turn(player)
            else:
                self.player_turn(player)

        # Computer's turn
        self.computer_turn()

        # Determine the winner
        for player in self.players:
            if player.hand.get_value() > 21:
                print(f"{player.name} busted! Computer wins.")
            elif self.computer_player.hand.get_value() > 21:
                print(f"Computer busted! {player.name} wins.")
            elif player.hand.get_value() > self.computer_player.hand.get_value():
                print(f"{player.name} wins!")
            elif player.hand.get_value() < self.computer_player.hand.get_value():
                print(f"Computer wins. {player.name}, better luck next time!")
            else:
                print(f"{player.name} and Computer have a tie!")
                
    

In [13]:
card_counter_player = CardCounterPlayer("Card Counter Player")
game = BlackjackGame(num_players=1)
game.players.append(card_counter_player)
game.play_with_card_counter()

Player 1, do you want to 'hit' or 'stand'? hit
Player 1's Hand: 7 3 J
Player 1, do you want to 'hit' or 'stand'? stand
Player 1's Hand: 7 3 J
Card Counter Player's Hand: 8 2
Player 1 and Computer have a tie!
Computer wins. Card Counter Player, better luck next time!


7. Create a test scenario where one player, using the above strategy, is playing with a dealer and 3 other players that follow the dealer's strategy. Each player starts with same number of chips. Play 50 rounds (or until the strategy player is out of money). Compute the strategy player's winnings. You may remove unnecessary printouts from your code (perhaps implement a verbose/quiet mode) to reduce the output.

8. Create a loop that runs 100 games of 50 rounds, as setup in previous question, and store the strategy player's chips at the end of the game (aka "winnings") in a list. Histogram the winnings. What is the average winnings per round? What is the standard deviation. What is the probabilty of net winning or lossing after 50 rounds?


9. Repeat previous questions scanning the value of the threshold. Try at least 5 different threshold values. Can you find an optimal value?

10. Create a new strategy based on web searches or your own ideas. Demonstrate that the new strategy will result in increased or decreased winnings. 